In [1]:
from src import gravix2
import tempfile

In [2]:
import matplotlib.pyplot as plt
import numpy as np

import nbfigtulz as ftl

In [3]:
img_dir = tempfile.TemporaryDirectory()
ftl.config['img_dir'] = img_dir.name
ftl.config['img_dir']

'/tmp/tmp9c9cs8cq'

In [4]:
def shoot(missile, *, n=1, h=1e-3):
    x = np.empty((0, 3))
    v = np.empty((0, 3))
    
    premature = False
    while n > 0 and not premature:
        n -= 1
        premature = missile.propagate(planets, h=h)
        if premature:
            print('!!! PREMATURE !!!')
        
        x = np.concatenate((x, missile.trajectory.x), axis=0)
        v = np.concatenate((v, missile.trajectory.n * missile.trajectory.v[:, np.newaxis]), axis=0)
    
    return x, v

In [5]:
libgravix2 = gravix2.load_library('libs/libgravix2.so')
planets = libgravix2.new_planets([(0., 0.)])
missiles = libgravix2.new_missiles(1)

In [6]:
libgravix2.config

Config(pot_type='2D', n_pot=None, trajectory_size=100, int_steps=10, min_dist=1.0, p_min=0.0001, composition_scheme='p8s15')

In [7]:
libgravix2.v_esc

249.5199777150688

In [8]:
libgravix2.estimate_orb_period(v0=2. * libgravix2.v_esc, h=1e-3)

81.12749662792896

In [9]:
m = missiles[0]
m.launch(planets=planets, planet_idx=0, v=libgravix2.v_esc, psi=0.)

x, v = shoot(m, n=50)
x.shape, v.shape

((5000, 3), (5000, 3))

In [10]:
@ftl.with_context
def make_fig(x, label):
    fig, ax = plt.subplots()
    ax.set_ylabel(label)
    ax.plot(np.rad2deg(x), '.', alpha=.2)
        
    return ftl.save_fig(fig, label)
        
        
ftl.img_grid([
    make_fig(np.arcsin(x[:, 2]), 'Latitude'),
    make_fig(np.arctan2(x[:, 0], x[:, 1]), 'Longitude'),
])

In [11]:
m = missiles[0]
m.launch(planets=planets, planet_idx=0, v=2. * libgravix2.v_esc, psi=-90.)

x, v = shoot(m, n=1)
x.shape, v.shape

!!! PREMATURE !!!


((82, 3), (82, 3))

In [12]:
ftl.img_grid([
    make_fig(np.arcsin(x[:, 2]), 'Latitude'),
    make_fig(np.arctan2(x[:, 0], x[:, 1]), 'Longitude'),
])

In [13]:
libgravix2 = gravix2.load_library('libs/libgravix2.so')
planets = libgravix2.new_planets([(10., 20.), (-20., -30)])
missiles = libgravix2.new_missiles(1)

In [14]:
m = missiles[0]
m.launch(planets=planets, planet_idx=0, v=200., psi=90.)

x, v = shoot(m, n=3)
x.shape, v.shape

((300, 3), (300, 3))

In [15]:
m = missiles[0]
m.set(pos=x[-1], orientation=-v[-1], v=np.sqrt(np.sum(v[-1] ** 2)))

x2, v2 = shoot(m, n=3)
x2.shape, v2.shape

((300, 3), (300, 3))

In [16]:
@ftl.with_context
def make_fig(x, planets):
    lat = np.rad2deg(np.arcsin(x[:, 2]))
    lon = np.rad2deg(np.arctan2(x[:, 0], x[:, 1]))

    fig, ax = plt.subplots()
    ax.plot(lon, lat)

    ax.grid()
    ax.set_xlabel('Longitude (deg.)')
    ax.set_ylabel('Latitude (deg.)')

    for plat, plon in planets:
        ax.plot(plon, plat, 'o', c='black', alpha=.5)
        
    return ftl.save_fig(fig, 'trajectories')
        
        
ftl.img_grid([
    make_fig(x, planets.planet_pos),
    make_fig(x2, planets.planet_pos),
])

,


In [17]:
@ftl.with_context
def make_fig(x1, x2):
    lat1 = np.rad2deg(np.arcsin(x1[:, 2]))
    lon1 = np.rad2deg(np.arctan2(x1[:, 0], x1[:, 1]))
    
    lat2 = np.rad2deg(np.arcsin(x2[:, 2]))
    lon2 = np.rad2deg(np.arctan2(x2[:, 0], x2[:, 1]))

    fig, ax = plt.subplots()
    ax.plot(lat1 - lat2[::-1], label=r'$\Delta \phi$')
    ax.plot(lon1 - lon2[::-1], label=r'$\Delta \lambda$')
    
    ax.legend()
    ax.set_xlabel('Time step')
    ax.set_ylabel('Error')
    
    return ftl.save_fig(fig, 'diff')
    
    
make_fig(x[:-1], x2[:-1])

diff.png

In [18]:
libgravix2 = gravix2.load_library('libs/libgravix2.so')
planets = libgravix2.new_planets([(0., 0.),])
missiles = libgravix2.new_missiles(1)

r = 10.
v_scrcl = libgravix2.v_scrcl(10.)
print(f'{v_scrcl=}')
m = missiles[0]
m.set(pos=(r, 0.), orientation=(0., 1.), v=v_scrcl)
x, v = shoot(m, n=10)
x.shape, v.shape

v_scrcl=81.34036685115933


((1000, 3), (1000, 3))

In [19]:
@ftl.with_context
def make_fig(x, v, r_gt, v_gt):
    lat = np.arcsin(x[:, 2])
    lon = np.arctan2(x[:, 0], x[:, 1])
    dist = np.arccos(np.cos(lat) * np.cos(lon))
    v_abs = np.sqrt(np.sum(v**2, axis=1))
    
    r_error = np.rad2deg(dist - np.deg2rad(r_gt))
    v_error = v_abs - v_gt

    fig, ax = plt.subplots()
    ax.plot(r_error, label='$r-$GT (deg.)')
    ax.plot(v_error, label='$v-$GT')

    ax.grid()
    ax.set_xlabel('Time step')
    ax.set_ylabel('Error')
    ax.legend()

    return ftl.save_fig(fig, 'orbit_error')


make_fig(x, v, r, v_scrcl)

orbit_error.png